# Projet Quadratic Assignment Problem

*Antoine PERRIN & Matthieu CAYET*

In [1]:
# Appelles techniques

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


## Jeux de données

Pour ce projet, nous avons utilisé les [instances "Tai..a" de Taillard](http://anjos.mgi.polymtl.ca/qaplib/inst.html#Ta). Chaque instance est caractérisé par:
* un nombre *n* équivalent à la taille de l'instance
* une matrice de distances *A*
* une matrice de flux (ou de poids) *B*

Nous utilisons les instances suivantes:
* Tai12a
* Tai15a
* Tai17a
* Tai20a
* Tai25a
* Tai30a
* Tai35a
* Tai40a
* Tai50a
* Tai60a
* Tai80a
* Tai100a

### Exemple d'instance Taillard 12

In [2]:
import pandas as pd
from Models.TaillardParser import TaillardParser as tp

tai12a = tp('tai12a.dat')

#### Matrice de distance

In [3]:
pd.DataFrame(tai12a.get_distance_matrix())

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,27,85,2,1,15,11,35,11,20,21,61
1,27,0,80,58,21,76,72,44,85,94,90,51
2,85,80,0,3,48,29,90,66,41,15,83,96
3,2,58,3,0,74,45,65,40,54,83,14,71
4,1,21,48,74,0,77,36,53,37,26,87,76
5,15,76,29,45,77,0,91,13,29,11,77,32
6,11,72,90,65,36,91,0,87,67,94,79,2
7,35,44,66,40,53,13,87,0,10,99,56,70
8,11,85,41,54,37,29,67,10,0,99,60,4
9,20,94,15,83,26,11,94,99,99,0,56,2


#### Matrice de flux

In [4]:
pd.DataFrame(tai12a.get_connexion_matrix())

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,21,95,82,56,41,6,25,10,4,63,6.0
1,21,0,44,40,75,79,0,89,35,9,1,85.0
2,95,44,0,84,12,0,26,91,11,35,82,26.0
3,82,40,84,0,69,56,86,45,91,59,18,76.0
4,56,75,12,69,0,39,18,57,36,61,36,21.0
5,41,79,0,56,39,0,71,11,29,82,82,6.0
6,6,0,26,86,18,71,0,71,8,77,74,30.0
7,25,89,91,45,57,11,71,0,89,76,76,40.0
8,10,35,11,91,36,29,8,89,0,93,56,1.0
9,4,9,35,59,61,82,77,76,93,0,50,4.0


## Voisinage

Afin de varier les résultats possibles, nous avons implémenter plusieurs manières de calculer le voisinage d'un élement.

*Par soucis de compéhension, l'ensemble des exemples de cette partie se font sur l'instance Tai12a.*

In [5]:
from Services.Voisinage import Voisinage as vn

### Voisinage par distance

Ce fut notre première manière d'implémenter le voisinage.

Son principe est simple. Nous intervertissons uniquement les éléments dont la distance est inférieur à la distance maximale définie.

La méthode `get_permutations_by_distances` nous permet de calculer à l'avance la liste des emplacements à échanger à chaque nouveau calcul de voisinage.

#### Exemple

In [6]:
vn_distance = vn(tai12a.get_distance_matrix(),"distances",1)
def show_vn_distance(vn,distance):
    df = pd.DataFrame(vn.get_permutations_by_distances(vn.distances, distance), columns=['a','b'])
    print(df.to_string(index=False))

display(interactive(show_vn_distance, vn=fixed(vn_distance), distance=(1,100)))

interactive(children=(IntSlider(value=50, description='distance', min=1), Output()), _dom_classes=('widget-int…

Nous récupérons ainsi la liste des éléments ayant une distance inférieure à la distance sélectionnée.

#### Avantages

Le gros avantage de cette technique est qu'elle rapide puisqu'elle necessite de parcour la matrice de distance qu'une seule fois.

De plus, il est facile ce mode de sélection puisque nous cherchons la liste des emplacements "physiquements" proches.

#### Inconvénients

Nous avons trouvé plusieurs inconvénients à cette méthode.

La première est qu'il se peut que des emplacements ne soient jamais sélectionner si leur distance minimale est supérieur à la distance sélectionnée.

Ensuite, avec un nombre plus importants de emplacements, notre tableau risque de grossir énormément vite.

### Voisinage aléatoire

Nous tirons aléatoirement une liste de permutations parmit toutes les permutations de l'instance.

#### Exemple

In [9]:
vn_random = vn(tai12a.get_distance_matrix(),"random",1)

def show_vn_random(vn,taille):
    df = pd.DataFrame(vn.get_permutations_random(vn.distances, taille), columns=['a','b'])
    print(df.to_string(index=False))

display(interactive(show_vn_random, vn=fixed(vn_random), taille=(1,65)))

interactive(children=(IntSlider(value=33, description='taille', max=65, min=1), Output()), _dom_classes=('widg…

#### Avantages

Chaque emplacements à la meme probabilité d'apparaitre dans une liste de voisins meme s'il est isolé des autres emplacements.

#### Inconvénients

Fonctionnement non déterministe, nous ne pouvons pas assurer qu'un emplacement soit visiter au moins une fois.

### Voisinage complet

Nous récupérons l'ensemble des possiblités de permutations de notre instance.

#### Exemple

In [13]:
vn_default = vn(tai12a.get_distance_matrix())

def show_vn_default(vn):
    df = pd.DataFrame(vn.get_permutations(vn.distances), columns=['a','b'])
    print(df.to_string(index=False))

display(interactive(show_vn_default, vn=fixed(vn_default)))

interactive(children=(Output(),), _dom_classes=('widget-interact',))

#### Avantages

Cette façon de calculer les voisins est déterministe. Nous sommes sur que l'ensemble des voisins soient visiter.

#### Inconvénients

Cette façon de faire est lourde et la liste grandit de façon exponentielle suivant le nombre d'emplacements total.

In [8]:
voisinage = Voisinage(distances)
x = voisinage.get_random_x()
fitness = Fitness(connexions, distances)
tabou = Tabou(distances, connexions, voisinage, fitness)
print(x)

NameError: name 'Voisinage' is not defined

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from Services.Tabou import Tabou as tb
# out = interactive_output(voisinage.get_random_x)
voisinage = vn(distances)

interact_manual(tb.resolve, x=fixed(voisinage.get_random_x()), tabousize=range(0,10), maxIter=range(0,1000,100))

print("solution optimale: ")
print([7,0,5,1,10,9,2,4,8,6,11,3])
print("Fitness optimale: ")
print(fitness.calcul([7,0,5,1,10,9,2,4,8,6,11,3]))

In [ ]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)